In [1]:
import pandas as pd
import numpy as np
import os
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score

In [49]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV


In [2]:
df_train = pd.read_csv("D:/online dataset csv files preprocessing/MyDataset_sw_bp_same_train.csv")
df_test = pd.read_csv("D:/online dataset csv files preprocessing/MyDataset_sw_bp_same_test.csv")

In [4]:
df_train.shape, df_test.shape

((229190, 8), (98246, 8))

In [15]:
X_train= df_train[['LA(x)','LA(y)','LA(z)','GY(x)','GY(y)','GY(z)','label']].copy()
# y_train = df_train['label'].copy()

X_test= df_test[['LA(x)','LA(y)','LA(z)','GY(x)','GY(y)','GY(z)','label']].copy()
# y_test = df_test['label'].copy()

In [16]:
X_test

,LA(x),LA(y),LA(z),GY(x),GY(y),GY(z),label
0,1.742467,-0.241556,8.041262,0.100790,-1.040349,-1.389296,0
1,1.963214,0.196158,7.817154,0.166456,-0.965826,-1.434804,0
2,2.273200,0.602439,7.468016,0.197915,-0.858622,-1.491307,0
3,2.736319,0.958790,6.743418,0.187530,-0.728206,-1.545367,0
4,2.913777,1.012366,5.666056,0.133776,-0.591071,-1.585683,0
...,...,...,...,...,...,...,...
98241,0.032646,0.028275,0.519662,0.135150,-0.022907,0.223494,30
98242,-0.478584,-0.296353,0.069982,0.152254,-0.098041,0.268391,30
98243,-0.897290,-0.563466,-0.287334,0.155308,-0.165234,0.318786,30
98244,-1.035197,-0.655348,-0.474507,0.145534,-0.212269,0.362156,30


In [25]:
Fs = 200
frame_size = Fs*3
hop_size = Fs*1

In [33]:
num_columns = frame_size*6
new_df = pd.DataFrame(columns=range(num_columns))
for i in range(0,num_columns):
    if i in range(0,frame_size):
        string = f"LAx_{i+1}"
        new_df = new_df.rename(columns={i: string})
        
    if i in range(frame_size,frame_size*2):
        string = f"LAy_{i+1}"
        new_df = new_df.rename(columns={i: string}) 
       
    if i in range(frame_size*2,frame_size*3):
        string = f"LAz_{i+1}"
        new_df = new_df.rename(columns={i: string}) 
       
    if i in range(frame_size*3,frame_size*4):
        string = f"GYx_{i+1}"
        new_df = new_df.rename(columns={i: string})
        
    if i in range(frame_size*4,frame_size*5):
        string = f"GYy_{i+1}"
        new_df = new_df.rename(columns={i: string}) 
       
    if i in range(frame_size*5,frame_size*6):
        string = f"GYz_{i+1}"
        new_df = new_df.rename(columns={i: string}) 
new_df        

,LAx_1,LAx_2,LAx_3,LAx_4,LAx_5,LAx_6,LAx_7,LAx_8,LAx_9,LAx_10,...,GYz_3591,GYz_3592,GYz_3593,GYz_3594,GYz_3595,GYz_3596,GYz_3597,GYz_3598,GYz_3599,GYz_3600


In [71]:
new_df_train = new_df.copy()
new_df_test = new_df.copy()

In [77]:

def get_frames(scaled_X, frame_size, hop_size, df):
    last_value = scaled_X['label'].iloc[-1]
    for j in range(1, last_value + 1):
        filtered_df = scaled_X[scaled_X['label'] == j].drop('label', axis=1)
        for i in range(0, len(filtered_df) - frame_size, hop_size):
            subarray = filtered_df.iloc[i: i + frame_size].values
            if subarray.shape[0] != frame_size:
                continue
            label.append(j)
            merged_array = subarray.ravel()
            df.loc[len(df)] = merged_array    
    df['label'] = label

In [78]:
label = []
get_frames(X_train, frame_size, hop_size,new_df_train)

In [79]:
new_df_train.shape

(1056, 3601)

In [80]:
label = []
get_frames(X_test, frame_size, hop_size,new_df_test)

In [81]:
new_df_test.shape

(408, 3601)

In [82]:
new_df_test = new_df_test.astype('float32')


In [83]:
new_df_train = new_df_train.astype('float32')


In [84]:
df_train = new_df_train.sample(frac=1, random_state=0).reset_index(drop=True)

In [85]:
df_test = new_df_test.sample(frac=1, random_state=0).reset_index(drop=True)

In [86]:
df_test

,LAx_1,LAx_2,LAx_3,LAx_4,LAx_5,LAx_6,LAx_7,LAx_8,LAx_9,LAx_10,...,GYz_3592,GYz_3593,GYz_3594,GYz_3595,GYz_3596,GYz_3597,GYz_3598,GYz_3599,GYz_3600,label
0,1.696515,3.012438,3.908953,0.357091,0.144542,-0.410362,2.226851,2.992558,5.834365,0.466229,...,-0.366967,1.799403,-0.181803,-0.660756,-1.000393,1.853306,-0.346198,1.811315,-0.176306,20.0
1,-0.259373,-0.392220,0.318180,0.097480,0.096055,-0.040611,-0.259892,-0.392724,0.318264,0.097654,...,0.156148,0.004352,0.073378,-0.800133,0.091725,0.276177,0.161951,0.003436,0.083457,25.0
2,-0.003680,-0.069081,-0.139813,0.166863,0.142099,0.035111,0.188889,-0.193637,0.067443,0.167037,...,0.096004,-0.181726,0.358251,-0.257679,0.192050,0.070324,0.095945,-0.181708,0.358264,25.0
3,-0.546274,-0.502629,-0.172487,0.212346,0.259381,0.248691,-0.567287,-0.494932,-0.151527,0.214789,...,0.081421,-0.085239,-0.254240,-0.258561,-0.902648,-0.884156,0.057496,-0.089718,-0.237238,25.0
4,-1.219353,1.566666,1.074631,-0.211124,-1.011818,0.092925,-0.224944,2.416283,1.559083,-0.265490,...,0.558849,-0.646047,0.310998,-4.396692,-3.946408,6.285454,0.702093,-0.730344,0.349481,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403,0.030779,-0.007852,-0.007960,-0.072641,0.044350,-0.122204,0.030735,-0.007847,-0.007933,-0.072643,...,-0.073740,0.049908,-0.133116,0.004527,-0.004694,0.008410,-0.073742,0.049917,-0.133134,25.0
404,6.818722,-1.207684,-2.348887,-0.744699,0.413925,-0.913751,6.996873,-1.394519,-2.695269,-0.813420,...,-0.164089,-5.193500,1.474738,4.400823,-2.282772,-7.165644,-0.255309,-4.809582,1.233962,16.0
405,-3.608145,-7.703201,5.550821,0.616192,-0.415987,0.226815,-4.400406,-8.268530,5.870539,0.653657,...,0.111327,3.168847,0.324894,2.312674,-1.297724,1.809722,0.154086,3.193280,0.291909,10.0
406,0.196936,1.617004,0.625740,-0.513799,0.175695,-0.301682,0.124363,1.558055,0.570479,-0.515021,...,0.744623,-0.478217,0.127209,1.658495,-0.559981,-2.004943,0.710721,-0.514257,0.141258,4.0


In [97]:
X_train = df_train.drop("label", axis=1)
y_train = df_train['label']

X_test = df_test.drop("label", axis=1)
y_test = df_test['label']


In [98]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test)


In [89]:
def accuracy_metric(preds, dmatrix):
    labels = dmatrix.get_label()
    preds = np.argmax(preds, axis=1)  # Convert probabilities to class labels
    accuracy = (preds == labels).mean() * 100.0
    return 'accuracy', accuracy


In [111]:
params = {
    'max_depth': 7,
    'learning_rate': 0.3,
    'objective': 'multi:softmax',
    'num_class': 31,
    'subsample': 0.8,
    'alpha': 0.5,
    "gamma": 0
}
epochs = 15
evals_result = {}  # Initialize an empty dictionary to store the evaluation results

In [112]:
model = xgb.train(params, dtrain, epochs, evals=[(dtrain, 'train')],
                  evals_result=evals_result, verbose_eval=1)

[0]	train-mlogloss:2.55135
[1]	train-mlogloss:2.00453
[2]	train-mlogloss:1.59610
[3]	train-mlogloss:1.29390
[4]	train-mlogloss:1.06140
[5]	train-mlogloss:0.88254
[6]	train-mlogloss:0.74050
[7]	train-mlogloss:0.62662
[8]	train-mlogloss:0.53676
[9]	train-mlogloss:0.46279
[10]	train-mlogloss:0.40124
[11]	train-mlogloss:0.34981
[12]	train-mlogloss:0.30990
[13]	train-mlogloss:0.27477
[14]	train-mlogloss:0.24735


In [113]:
predictions = model.predict(dtest)

In [114]:
X_test.shape

(408, 3600)

In [115]:
y_test.shape

(408,)

In [116]:
predictions.shape


(408,)

In [117]:
accuracy_score(y_test, predictions)

0.34068627450980393